In [1]:
import numpy as np
import matplotlib.pyplot as plt 

# Librería de árboles

In [2]:
# coding=utf-8
# para crear copias de los arboles
from copy import deepcopy


# esta funcion dice si el argumento es una funcion o no
def is_function(f):
    return hasattr(f, "__call__")


# esto nos permite recorrer una lista de a pedazos
# ejemplo el input es [1,2,3,4,5,6,7,8]
# con n=2 esta funcion nos hara tener
# [(1,2), (3,4), (5,6), (7,8)]
def chunks(iterable, n):
    for i in range(0, len(iterable), n):
        yield iterable[i:i + n]


class Node:
    def __init__(self, function):
        # nos aseguramos que el nodo recibe una funcion
        assert is_function(function)
        self.operation = function
        # esto nos permite contar cuantos argumentos recibe la funcion
        self.num_arguments = function.__code__.co_argcount
        self.arguments = []
        

    # funcion para evaluar un nodo (calcular el resultado)
    def eval(self, vals=None):
        # es importante chequear que los argumentos que nos dieron
        # coincidan con los argumentos que necesitamos
        assert len(self.arguments) == self.num_arguments
        # evaluamos los argumentos, y luego los pasamos como argumentos a
        # la operacion
        # NOTA: esto -> *[...] significa que separa los elementos de la lista
        # ejemplo si tenemos `lista = [1,2,3]`
        # print(lista) -> [1,2,3]
        # print(*lista) -> 1 2 3
        # esto se llama `unpacking`.
        # lo necesitamos porque nuestra funcion recibe N argumentos
        # no una lista de tamaño N.
        if vals is None:
            return self.operation(*[node.eval() for node in self.arguments])
        else:
            return self.operation(*[node.eval(vals) for node in self.arguments])        

   
    # hace una lista con todos los hijos
    def serialize(self):
        l = [self]
        for node in self.arguments:
            l.extend(node.serialize())
        return l

    # copia al nodo
    def copy(self):
        return deepcopy(self)

    # reemplaza el nodo por otro
    def replace(self, otherNode):
        # aqui estamos haciendo algo medio hacky.
        # la forma correcta seria tener una referencia al nodo padre y
        # reemplazarse a si mismo por otro.
        # por temas de mantener el codigo corto lo hacemos asi
        # pero no lo hagan en casa!
        assert isinstance(otherNode, Node)
        self.__class__ = otherNode.__class__
        self.__dict__ = otherNode.__dict__


# esta clase representa todos los nodos quetienen 2 argumentos
class BinaryNode(Node):
    num_args = 2

    def __init__(self, function, left, right):
        # revisamos que todo sea un nodo y agregamos a las lista de
        # argumentos
        assert isinstance(left, Node)
        assert isinstance(right, Node)
        super(BinaryNode, self).__init__(function)
        self.arguments.append(left)
        self.arguments.append(right)
        

class AddNode(BinaryNode):
    def __init__(self, left, right):
        def _add(x, y):
            return x + y
        super(AddNode, self).__init__(_add, left, right)
    

    # esta es la funcion que define como se mostrara el nodo
    # como es un nodo que REPResenta la suma, lo mostramos como suma
    def __repr__(self):
        return "({} + {})".format(*self.arguments)


class SubNode(BinaryNode):
    def __init__(self, left, right):
        def _sub(x, y):
            return x - y

        super(SubNode, self).__init__(_sub, left, right)
        

    def __repr__(self):
        return "({} - {})".format(*self.arguments)


class MaxNode(BinaryNode):
    def __init__(self, left, right):
        def _max(x, y):
            return max(x, y)

        super(MaxNode, self).__init__(_max, left, right)
        

    def __repr__(self):
        return "max({{{}, {}}})".format(*self.arguments)


class MultNode(BinaryNode):
    def __init__(self, left, right):
        def _mult(x, y):
            return x * y

        super(MultNode, self).__init__(_mult, left, right)
       

    def __repr__(self):
        return "({} * {})".format(*self.arguments)


class DivNode(BinaryNode):
    def __init__(self, left, right):
        def _div(x, y):
            return x / y
            
        super(DivNode, self).__init__(_div, left, right)
        

    def __repr__(self):
        return "({} / {})".format(*self.arguments)


class TerminalNode(Node):
    # Este nodo representa una hoja de arbol. Es el nodo terminal
    # por lo que no tiene argumentos
    num_args = 0

    def __init__(self, value):
        # igual tenemos que representarlo como una funcion, por como
        # diseñamos el programa. Pero aqui va a ser una funcion vacia
        def _nothind(): pass

        super(TerminalNode, self).__init__(_nothind)
        self.value = value
        

    def __repr__(self):
        return str(self.value)

    def eval(self, vals=None):
        try:
            return float(self.value)
        except :
            return vals[self.value]


# Librería generadora aleatoria de árboles

In [3]:
class AST:
    def __init__(self, allowed_functions, allowed_terminals, prob_terminal=0.3):
        
        # las funciones (nodos en nuestro caso) que nuestro programa puede tener
        self.functions = allowed_functions
        # los terminales admitidos en nuestro programa. Numeros por ejemplo
        self.terminals = allowed_terminals
        # para no tener un arbol infinitamente profundo, existe una posibilidad
        # de que, a pesar de que ahora toque hacer otro sub arbol, que se ignore
        # eso y se ponga un terminal en su lugar.
        self.prob = prob_terminal

    # esta funcion ya la hemos visto, nos permite llamar al AST como si fuera
    # una funcion. max_depth es la produndidad que queremos tenga el arbol
    def __call__(self, max_depth=10):
        
        # aqui tenemos una funcion auxiliar. Nos permitira hacer esto recursivo
        def create_rec_tree(depth):
            # si `depth` es mayor a 0, nos toca crear un sub-arbol
            if depth > 0:
                # elegimos una funcion aleatoriamente
                node_cls = np.random.choice(self.functions)
                # aqui iremos dejando los argumentos que necesita la funcion
                arguments = []
                # para cada argumento que la funcion necesite...
                for _ in range(node_cls.num_args):
                    # existe un `prob` probabilidad de que no sigamos creando
                    # sub-arboles y lleguemos y cortemos aqui para hacer
                    # un nodo terminal
                    if np.random.random() < self.prob:
                        arguments.append(create_rec_tree(0))
                    else:
                        # la otra es seguir creando sub-arboles recursivamente
                        arguments.append(create_rec_tree(depth - 1))
                
                # `arguments` es una lista y los nodos necesitan argumentos
                # asi que hacemos "unpacking" de la lista
                return node_cls(*arguments)
            else:
                # si `depth` es 0 entonces creamos un nodo terminal con
                # alguno de los terminales permitidos que definimos inicialmente
                return TerminalNode(np.random.choice(self.terminals))

        # llamamos a la funcion auxiliar para crear un arbol de profundidad `max_depth`
        return create_rec_tree(max_depth)

# Librería del algoritmo genético modificado

In [4]:
# coding=utf-8
import numpy as np
from numpy import random



# Definimos una clase GENALG para nuestro algoritmo genético
class GENALG(object):

    # Método inicializador para la clase algoritmo genético : recibe 8 inputs, 2 de ellos opcionales
    # - Un número entero positivo pop_sz_0 con el tamaño de población
    # - Una función fit_fn que permite obtener el fitness de un individuo
    # - Una función cr_genes que permite crear los genes de un individuo
    # - Una función cr_indv que permite crear a los individuos de la población
    # - Un diccionario de diccionarios indv_chars que para cada gen del individuo posee un diccionario con información
    #   acerca del tipo de gen y del conjunto de posibles valores que este puede tomar este gen.
    # - Un diccionario term_cond que entrega información sobre el criterio de parada, específicamente,
    #   el tipo de criterio a ocupar ('iterations' o 'fitness_th'), un valor de fitness a alcanzar fitness_th (No
    #   necesariamente el óptimo) y el número de iteraciones máximo.
    # - (Opcional) Tasa de mutación mut_rate (float entre 0.0 y 1.0) que da la probabilidad de que un gen de un
    #    individuo tenga una mutación (por defecto toma el valor 0.2)
    # - (Opcional) Tasa de elitismo elitism_rate (float entre 0.0 y 1.0) que da el porcentaje de la población que será
    #    ocupado para conservar a los mejores individuos de la generación pasada (por defecto toma el valor 0.0,
    #    esto es, por defecto no se ocupa elitismo)
    def __init__(self, pop_sz_0: int, fit_fn, cr_genes, cr_indv, indv_chars, term_cond, mut_rate=0.2, elitism_rate=0.0):
        # Verificamos que los parámetros de entrada del constructor sean los correctos
        if type(pop_sz_0) != int:
            raise ValueError("Input n_weights debe ser un número entero positivo")
        # atributo con el tamaño de la población
        self.__pop_sz = pop_sz_0
        # atributo con un diccionario de la población actual
        self.__pop = None
        # atributo con un diccionario del fitness de cada individuo de la población actual
        self.__pop_fitness = None
        # atributo con el fitness máximo dentro de la población actual
        self.__max_fitness = None
        # atributo con el fitness promedio de la población actual
        self.__mean_fitness = None
        # atributo con el fitness mínimo dentro de la población actual
        self.__min_fitness = None
        # atributo con el individuo con el fitness máximo dentro de la población actual
        self.__fittest_indv = None
        # atributo con la función de fitness
        self.__fit_fn = fit_fn
        # atributo con la función de creación de genes
        self.__cr_genes = cr_genes
        # atributo con la función de creación de individuos
        self.__cr_indv = cr_indv
        # atributo con el diccionario que caracteriza a un individuo
        self.__indv_chars = indv_chars        
        # atributo con el diccionario del criterio de parada
        self.__term_cond = term_cond
        # atributo con la tasa de mutación
        self.__mut_rate = mut_rate
        # atributo con la cantidad de individuos para aplicar elitismo
        self.__elitism = int(np.ceil(elitism_rate * pop_sz_0))

        # Métodos get para obtener los atributos del algoritmo genético.

    # Función para obtener el tamaño de la población
    def get_pop_sz(self):
        return self.__pop_sz

    # Función para obtener la población
    def get_pop(self):
        if self.__pop is None:
            raise ValueError("la población no ha sido creada")
        else:
            return self.__pop

    # Función para obtener la tasa de mutación
    def get_mut_rate(self):
        return self.__mut_rate

    # Función para obtener condición de terminación
    def get_term_cond(self):
        return self.__term_cond

        # Función para obtener la función de fitness del algoritmo

    def get_fit_fn(self):
        return self.__fit_fn

    # Función para obtener la función de creación de genes del algoritmo
    def get_cr_genes(self):
        return self.__cr_genes

    # Función para obtener la función de creación de individuos del algoritmo
    def get_cr_indv(self):
        return self.__cr_indv

    # Función para obtener el número de individuos al aplicar elitismo
    def get_etilitsm(self):
        return self.__elitism

    # Función para obtener el fitness de la población
    def get_pop_fitness(self):
        if self.__pop_fitness is None:
            raise ValueError("El fitness de la población no ha sido calculado")
        else:
            return self.__pop_fitness

    # Función para obtener el fitness máximo de la población
    def get_max_fitness(self):
        if self.__max_fitness is None:
            raise ValueError("El fitness de la población no ha sido calculado")
        else:
            return self.__max_fitness

    # Función para obtener el fitness mínimo de la población
    def get_min_fitness(self):
        if self.__min_fitness is None:
            raise ValueError("El fitness de la población no ha sido calculado")
        else:
            return self.__min_fitness

    # Función para obtener el fitness promedio de la población
    def get_mean_fitness(self):
        if self.__mean_fitness is None:
            raise ValueError("El fitness de la población no ha sido calculado")
        else:
            return self.__mean_fitness

    # Función para obtener el individuo con el fitness máximo de la población
    def get_fitness_indv(self):
        if self.__fitness_indv is None:
            raise ValueError("El fitness de la población no ha sido calculado")
        else:
            return self.__fitness_indv

    # Métodos para aplicar el algoritmo genético

    # Función para generar la población inicial
    def gen_pop(self):
        pop = {}  # Diccionario vacío para guardar la población
        # Generamos individuos acorde con el número de individuos de la población
        for i in range(self.__pop_sz):
            # Creamos un nuevo individuo con la función de creación de individuos, notar que la función recibe la
            # función de creación de genes y el diccionario de características de un individuo.
            pop[i] = self.__cr_indv(self.__cr_genes, self.__indv_chars)
        self.__pop = pop  # se actualiza el atributo con la población

    # Función para calcular el fitness de la población. Recibe un diccionario con parámetros especificos de la
    # función de fitness.
    def comp_pop_fitness(self, fit_params):
        pop_fitness = {}  # Diccionario vacío para guardar el fitness de la población
        new_pop = {}  # Diccionario vacío para guardar la población ordenada de manera descendiente en fitness
        l_pfitness = []  # lista para guardar los fitness de la población
        # Calculamos el fitness de cada individuo de la población y lo guardamos en una lista
        for i in range(self.__pop_sz):
            fitness = self.__fit_fn(self.__pop[i], fit_params)
            l_pfitness.append(fitness)
        mean_fitness = np.mean(l_pfitness)  # calculamos fitness promedio
        max_fitness = max(l_pfitness)  # calculamos fitness máximo
        min_fitness = min(l_pfitness)  # calculamos fitness mínimo
        fittest_indv = self.__pop[l_pfitness.index(max_fitness)].copy()  # copiamos el individuo con mayor fitness
        # actualizamos atributos de la clase
        self.__max_fitness = max_fitness
        self.__min_fitness = min_fitness
        self.__mean_fitness = mean_fitness
        self.__fittest_indv = fittest_indv
        # creamos una copia de la lista con los fitness para hacer operaciones
        l_aux = l_pfitness[:]
        # Generamos los diccionarios con la población ordenada de manera descendiente en fitness y con sus fitness
        for i in range(self.__pop_sz):
            max_ind = l_aux.index(max(l_aux))  # buscamos el lugar correspondiente al individuo con fitness máximo
            new_pop[i] = self.__pop[max_ind].copy()  # guardamos al individuo del lugar max_ind en su nueva posición
            pop_fitness[i] = l_aux[max_ind]  # guardamos el fitness del individuo en su nuevo lugar
            # actualizamos el valor de fitness del ind. en la lista para no volver a considerarlo
            l_aux[max_ind] = -np.inf
        # actualizamos la población y el fitness
        self.__pop = new_pop.copy()
        self.__pop_fitness = pop_fitness.copy()

    # Método para aplicar la selección mediante torneo de los individuos de la población. Recibe de entrada el número 
    # de competidores (entero positivo). Asume que la poblacíon está ordenada en base al fitness de manera
    # decreciente (paso que ocurre al calcular el fitness de la población con el método comp_pop_fitness)
    def tournament_sel(self, slots: int):
        if slots > self.__pop_sz:
            raise ValueError("slots debe ser un número entero positivo menor que el tamaño de la población")
        else:
            # elegimos un conjunto de índices al azar de tamaño igual al número de competidores
            chosen_set = np.random.randint(0, self.__pop_sz, slots)
            # Como la población está ordenada el ganador es el menor de los índices del conjunto
            winner = min(chosen_set)
            return self.__pop[winner]  # retornamos al individuo ganador

    # Método para aplicar la selección mediante ruleta de los individuos de la población.
    def roulette_sel(self):
        fit_vals = list(self.__pop_fitness.values()) # lista con valores de fitness
        num_neg_vals = sum(n < 0 for n in fit_vals)
        # si hay fitness negativos debemos normalizar estos valores conservando las distancias
        if num_neg_vals > 0:
            min_val = abs(min(fit_vals))
            new_fit_vals = [min_val + item for item in fit_vals]
        else:
            new_fit_vals = fit_vals[:]
        total_fit = sum(new_fit_vals)  # calculamos la suma de los fitness
        # si todas las fitness son iguales a 0 retornamos un individuo al azar
        if total_fit == 0:
            idx = np.random.randint(0, self.__pop_sz)
            return self.__pop[idx]
        else:
            sum_fit = 0  # llevamos la cuenta parcial de la suma de los fitness
            th = total_fit * np.random.rand()  # obtenemos un punto aleatorio entre 0 y la suma total de fitness
            for i in range(self.__pop_sz):
                sum_fit += new_fit_vals[i]
                if sum_fit > th:  # retornamos al individuo en la posición que logra superar el umbral th
                    return self.__pop[i]

    # Método para aplicar la operación crossover a 2 individuos y generar un nuevo individuo.
    def crossover(self, indv1, indv2):
        new_element = indv1.copy()  # Hacemos una copìa del padre 1
        p2 = np.random.choice(indv2.serialize()).copy()  # Hacemos una copia de un sub-arbol del padre 2
        np.random.choice(new_element.serialize()).replace(p2) #Reemplazamos en un nodo al azar el sub-arbol obtenido del padre 2
        return new_element.copy()  # Retornamos una copia del hijo creado

    # Método para aplicar la operación mutation a 1 individuo y generar un nuevo individuo.
    def mutation(self, indv):
        # Vemos si la mutación ocurre
        if np.random.rand() > self.__mut_rate:
            return indv.copy()
        else:
            new_element = indv.copy() # Hacemos una copìa del individuo
            params = self.__indv_chars.copy() # Hacemos una copia de las caracteristicas de un individuo
            params['max_depth'] = np.random.randint(0, self.__indv_chars['max_depth']+1) #Generamos un tamaño al azar
            p2 = self.__cr_indv(self.__cr_genes, params) # Creamos un arbol aleatorio
            np.random.choice(new_element.serialize()).replace(p2) #Reemplazamos en un nodo al azar del individuo el arbol obtenido p2
            return new_element.copy()  # Retornamos una copia del individuo mutado

    # Método para aplicar el algortimo genético. Recibe los parametros de la función de fitness en fit_params, el tipo
    # de selección selection_type a ocupar ('tournament' o 'roulette', por defecto tournament), una semilla para la
    # reproducibilidad de los resultados y la cantidad de ompetidores de la seleccion tournament en caso de necesitarlo
    # (por defecto 3)
    def apply(self, fit_params, selection_type='tournament', random_state=None, slots=3):
        if type(random_state) == int:
            np.random.seed(random_state)  # fijamos la semilla de ser ingresada
        self.gen_pop()   # generamos la población inicial
        self.comp_pop_fitness(fit_params)  #calculamos el fitness de la población inicial
        generations = {}  # gereramos un diccionario vacío para ir guardando la información de cada generación
        iters = 0  # fijamos el contador de generaciones en 0
        overall_max_fitness = self.__max_fitness  # inicializamos el registro del fitness máximo alcanzado
        overall_fittest_indv = self.__fittest_indv  # inicializamos el registro del individuo de máximo fitness
        generations[iters] = {}  # generamos un diccionario vacío para guardar la información de la generación
        generations[iters]['max_fitness'] = self.__max_fitness  # registramos fitness máximo de la generación actual
        generations[iters]['min_fitness'] = self.__min_fitness  # registramos fitness mínimo de la generación actual
        generations[iters]['mean_fitness'] = self.__mean_fitness  # registramos fitness promedio de la generación actual
        if self.__term_cond['type'] == 'iterations':  # Si el criterio de parada es un número de iteraciones máximo
            # fijamos el contador de la generación en la cual se alcanza por primera vez la meta de fitness en el doble
            # de la cantidad de iteraciones a realizar para distinguir los casos en los que no se alcanza la meta
            goal_cross = 2 * self.__term_cond['iters']
            # actualizamos el valor del contador de la generación de cruce de meta en caso que corresponda
            if self.__max_fitness >= self.__term_cond['fitness_th'] and goal_cross == 2 * self.__term_cond['iters']:
                goal_cross = iters
            while iters < self.__term_cond['iters']:  # iteramos hasta llegar al número de iteraciones máximo
                new_pop = {}  # generamos diccionario vacío para guardar nueva población
                # copiamos los mejores individuos de la generacion anterior de acuerdo a lo establecido en nuestra tasa
                # de elitismo
                for i in range(0, self.__elitism):
                    new_pop[i] = self.__pop[i].copy()
                # Los cupos restantes los producimos mediante reproducción
                for i in range(self.__elitism, self.__pop_sz):
                    # si elegimos la selección roulette sorteamos 2 padres mediante este método
                    if selection_type == 'roulette':
                        parent_1 = self.roulette_sel()
                        parent_2 = self.roulette_sel()
                    # en caso contrario sorteamos 2 padres mediante el método de selección tournament
                    else:
                        parent_1 = self.tournament_sel(slots)
                        parent_2 = self.tournament_sel(slots)
                    # creamos un hijo mediante crossover de estos 2 padres
                    child = self.crossover(parent_1, parent_2)
                    # Si la tasa de mutación es mayor a 0 aplicamos la operacion de mutacion al hijo para obtener los
                    # valores finales de sus genes
                    if self.__mut_rate > 0.0:
                        child = self.mutation(child)
                    new_pop[i] = child.copy()  # añadimos al hijo a la nueva población
                self.__pop = new_pop.copy()  # actualizamos la poblacion
                self.comp_pop_fitness(fit_params)  # recalculamos el fitness de la poblacion
                iters += 1  # aumentamos en 1 el contador de generación
                generations[iters] = {}  # generamos un diccionario vacío para guardar la información de la generación
                # registramos fitness máximo de la generación actual
                generations[iters]['max_fitness'] = self.__max_fitness
                # registramos fitness mínimo de la generación actual
                generations[iters]['min_fitness'] = self.__min_fitness
                # registramos fitness promedio de la generación actual
                generations[iters]['mean_fitness'] = self.__mean_fitness
                # Si se encontró un individuo que supere el máximo fitness alcanzado en generaciones anteriores
                # se actualiza el registro del mejor individuo y del valor de fitness máximo
                if self.__max_fitness > overall_max_fitness:
                    overall_max_fitness = self.__max_fitness
                    overall_fittest_indv = self.__fittest_indv
                # actualizamos el valor del contador de la generación de cruce de meta en caso que corresponda
                if self.__max_fitness >= self.__term_cond['fitness_th'] and goal_cross == 2 * self.__term_cond['iters']:
                    goal_cross = iters
        else: # en caso contrario, es decir, el criterio de parada es un umbral para el fitness
            while overall_max_fitness < self.__term_cond['fitness_th']:  # iteramos hasta superar el umbral de fitness
                new_pop = {}  # generamos diccionario vacío para guardar nueva población
                # copiamos los mejores individuos de la generacion anterior de acuerdo a lo establecido en nuestra tasa
                # de elitismo
                for i in range(0, self.__elitism):
                    new_pop[i] = self.__pop[i].copy()
                # Los cupos restantes los producimos mediante reproducción
                for i in range(self.__elitism, self.__pop_sz):
                    # si elegimos la selección roulette sorteamos 2 padres mediante este método
                    if selection_type == 'roulette':
                        parent_1 = self.roulette_sel()
                        parent_2 = self.roulette_sel()
                    # en caso contrario sorteamos 2 padres mediante el método de selección tournament
                    else:
                        parent_1 = self.tournament_sel(slots)
                        parent_2 = self.tournament_sel(slots)
                    # creamos un hijo mediante crossover de estos 2 padres
                    child = self.crossover(parent_1, parent_2)
                    # Si la tasa de mutación es mayor a 0 aplicamos la operacion de mutacion al hijo para obtener los
                    # valores finales de sus genes
                    if self.__mut_rate > 0.0:
                        child = self.mutation(child)
                    new_pop[i] = child.copy()  # añadimos al hijo a la nueva población
                self.__pop = new_pop.copy()  # actualizamos la poblacion
                self.comp_pop_fitness(fit_params)  # recalculamos el fitness de la poblacion
                iters += 1  # aumentamos en 1 el contador de generación
                generations[iters] = {}  # generamos un diccionario vacío para guardar la información de la generación
                # registramos fitness máximo de la generación actual
                generations[iters]['max_fitness'] = self.__max_fitness
                # registramos fitness mínimo de la generación actual
                generations[iters]['min_fitness'] = self.__min_fitness
                # registramos fitness promedio de la generación actual
                generations[iters]['mean_fitness'] = self.__mean_fitness
                # Si se encontró un individuo que supere el máximo fitness alcanzado en generaciones anteriores
                # se actualiza el registro del mejor individuo y del valor de fitness máximo
                if self.__max_fitness > overall_max_fitness:
                    overall_max_fitness = self.__max_fitness
                    overall_fittest_indv = self.__fittest_indv
                # Ponemos una condición adicional para terminar en caso que no encuentre a algún individuo que supere
                # el umbral de fitness
                if self.__term_cond['iters'] < iters:
                    break
            # actualizamos el valor del contador de la generación de cruce de meta en caso que corresponda
            if overall_max_fitness >= self.__term_cond['fitness_th']:
                goal_cross = iters
            else:
                goal_cross = 2 * self.__term_cond['iters']
        # retornamos la información de las generaciones, el valor del contador de la generación de cruce de meta,
        # el valor de fitness máximo y el registro del mejor individuo.
        return generations, goal_cross, overall_max_fitness, overall_fittest_indv

# Ejercicio 1

In [15]:
# Función para crear los genes de los individuos del ejercicio 1. Recibe 2 inputs:
# - gene_type: Corresponde al tipo de gen (En este caso es un bit por lo tanto es 'binary' )
# - fact_range: conjunto de valores posibles para un gen (En este caso los valores posibles son 0 o 1)
# Retorna un gen del tipo especificado y dentro del rango especificado.
def gene_factory_ex1(AST, indv_chars):
    allowed_functions = indv_chars['allowed_functions']
    allowed_terminals = indv_chars['allowed_terminals']
    prob_terminal = indv_chars['prob_terminal']
    factory = AST(allowed_functions, allowed_terminals, prob_terminal)    
    return factory

# Función para crear los individuos. Recibe 2 inputs:
# - Función creadora de genes: descrita arriba.
# - Diccionario caracterizando al individuo: diccionario que para cada gen del individuo contiene la información del
#   tipo de gen y del conjunto de valores posibles para ese gen
# Retorna un individuo adecuado para el problema.
def indv_factory(gene_factory_ex1, indv_chars): 
    factory = gene_factory_ex1(AST, indv_chars)
    max_depth = indv_chars['max_depth']
    new_indv = factory(max_depth)   
    return new_indv.copy()  # retornamos el individuo creado


# Función de fitness para el ejercicio 1. Recibe a un individuo y un diccionario con la secuencia y un ponderador.
# Retorna la multiplicación del ponderador y el valor absoluto de la diferencia entre el número entero que representa
# la secuencia entregada y el número entero que representa la secuencia del individuo.
def fitness_ex1(indv, objective):
    val = indv.eval()    
    fitness = objective['ponderador'] * abs(objective['secuencia'] - val)
    return -fitness

In [16]:
# Prueba ejercicio 1
# Seteamos los parametros a ser ocupados por el algoritmo para obtener los gráficos pedidos
secuencia_bits = 65346
ponderador = 1/100
fit_params = {'secuencia': secuencia_bits, 'ponderador': ponderador}
pop_sz_0 = 30
fit_fn = fitness_ex1
cr_genes = gene_factory_ex1
cr_indv = indv_factory
mut_rate = 0.1
term_cond = {'type': 'iterations', 'fitness_th': 0, 'iters': 200}
selection_type = 'tournament'
slots = 5
elitism_rate = 0.1
random_state = 42
indv_chars = {}
indv_chars['allowed_functions']=[AddNode,SubNode,MultNode,MaxNode]
indv_chars['allowed_terminals']= [25,7,8,100,4,2]
indv_chars['prob_terminal'] =0.3
indv_chars['max_depth']=10
GA = GENALG(pop_sz_0, fit_fn, cr_genes, cr_indv, indv_chars, term_cond, mut_rate, elitism_rate)
generations, goal_cross, overall_max_fitness, overall_fittest_indv = GA.apply(fit_params, selection_type,random_state, slots)
print(overall_max_fitness)
print(overall_fittest_indv)
# Obtener visualización de Evolución de fitness por generación sin elitismo
x = list(generations.keys())
vals = list(generations.values())
max_fit = [item['max_fitness'] for item in vals]
mean_fit = [item['mean_fitness'] for item in vals]
min_fit = [item['min_fitness'] for item in vals]

KeyboardInterrupt: 

In [26]:
fig, ax1 = plt.subplots(figsize=(15, 15))
ax1.set_xlabel("Generación", fontsize=15)
ax1.set_ylabel('Fitness (max )')
ax1.plot(x, max_fit, color='green', linewidth=2, label='max_fit')
ax1.legend(loc=2,fontsize=15)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Fitness (mean y min)')  # we already handled the x-label with ax1
ax2.plot(x,  mean_fit, color='blue', linewidth=2, label='mean_fit')
ax2.plot(x, min_fit, color='red', linewidth=2, label='min_fit')
ax2.legend(fontsize=15)
plt.title("Evolución de fitness por generación", fontsize=16, fontweight='bold')
plt.suptitle("Ejercicio 2.1.1", fontsize=20)

plt.savefig('Images\Fitness_por_Generacion_EX1.png')
plt.close()
print('gráfico 1 listo')

gráfico 1 listo


In [27]:
print(goal_cross)

24


# Ejercicio 2

In [28]:
# Función para crear los genes de los individuos del ejercicio 2. Recibe 2 inputs:
# - gene_type: Corresponde al tipo de gen (En este caso es un bit por lo tanto es 'binary' )
# - fact_range: conjunto de valores posibles para un gen (En este caso los valores posibles son 0 o 1)
# Retorna un gen del tipo especificado y dentro del rango especificado.
def gene_factory_ex1(AST, indv_chars):
    allowed_functions = indv_chars['allowed_functions']
    allowed_terminals = indv_chars['allowed_terminals']
    prob_terminal = indv_chars['prob_terminal']
    factory = AST(allowed_functions, allowed_terminals, prob_terminal)    
    return factory

# Función para crear los individuos. Recibe 2 inputs:
# - Función creadora de genes: descrita arriba.
# - Diccionario caracterizando al individuo: diccionario que para cada gen del individuo contiene la información del
#   tipo de gen y del conjunto de valores posibles para ese gen
# Retorna un individuo adecuado para el problema.
def indv_factory(gene_factory_ex1, indv_chars): 
    factory = gene_factory_ex1(AST, indv_chars)
    max_depth = indv_chars['max_depth']
    new_indv = factory(max_depth)   
    return new_indv.copy()  # retornamos el individuo creado


# Función de fitness para el ejercicio 1. Recibe a un individuo y un diccionario con la secuencia y un ponderador.
# Retorna la multiplicación del ponderador y el valor absoluto de la diferencia entre el número entero que representa
# la secuencia entregada y el número entero que representa la secuencia del individuo.
def fitness_ex2(indv, objective):
    val = indv.eval() 
    t_size=len(indv.serialize())
    fitness = objective['ponderador'][0] * abs(objective['secuencia'] - val)+objective['ponderador'][1] * t_size
    return -fitness

In [31]:
# Prueba ejercicio 2
# Seteamos los parametros a ser ocupados por el algoritmo para obtener los gráficos pedidos
secuencia_bits = 65346
ponderador = [1/100,3]
fit_params = {'secuencia': secuencia_bits, 'ponderador': ponderador}
pop_sz_0 = 30
fit_fn = fitness_ex2
cr_genes = gene_factory_ex1
cr_indv = indv_factory
mut_rate = 0.1
term_cond = {'type': 'iterations', 'fitness_th': 0, 'iters': 200}
selection_type = 'tournament'
slots = 5
elitism_rate = 0.1
random_state = 42
indv_chars = {}
indv_chars['allowed_functions']=[AddNode,SubNode,MultNode,MaxNode]
indv_chars['allowed_terminals']= [25,7,8,100,4,2]
indv_chars['prob_terminal'] =0.3
indv_chars['max_depth']=10
GA = GENALG(pop_sz_0, fit_fn, cr_genes, cr_indv, indv_chars, term_cond, mut_rate, elitism_rate)
generations, goal_cross, overall_max_fitness, overall_fittest_indv = GA.apply(fit_params, selection_type,random_state, slots)
print(overall_max_fitness)
print(overall_fittest_indv)
# Obtener visualización de Evolución de fitness por generación sin elitismo
x = list(generations.keys())
vals = list(generations.values())
max_fit = [item['max_fitness'] for item in vals]
mean_fit = [item['mean_fitness'] for item in vals]
min_fit = [item['min_fitness'] for item in vals]

C:\Users\pclev\Documents\DATA\GIT\Tarea_3\arboles.py:139: RuntimeWarning: overflow encountered in long_scalars
  return x * y
C:\Users\pclev\Documents\DATA\GIT\Tarea_3\arboles.py:115: RuntimeWarning: overflow encountered in long_scalars
  return x - y


-50.46
(8 * (100 + ((8 * (2 * 100)) + (8 * (8 * 100)))))


In [32]:
overall_fittest_indv.eval()

64800

In [33]:
fig, ax1 = plt.subplots(figsize=(15, 15))
ax1.set_xlabel("Generación", fontsize=15)
ax1.set_ylabel('Fitness (max )')
ax1.plot(x, max_fit, color='green', linewidth=2, label='max_fit')
ax1.legend(loc=2,fontsize=15)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Fitness (mean y min)')  # we already handled the x-label with ax1
ax2.plot(x,  mean_fit, color='blue', linewidth=2, label='mean_fit')
ax2.plot(x, min_fit, color='red', linewidth=2, label='min_fit')
ax2.legend(fontsize=15)
plt.title("Evolución de fitness por generación", fontsize=16, fontweight='bold')
plt.suptitle("Ejercicio 2.1.2", fontsize=20)

plt.savefig('Images\Fitness_por_Generacion_EX2.png')
plt.close()
print('gráfico 2 listo')

gráfico 2 listo


## Heatmap

# Ejercicio 3

In [44]:
# Función para crear los genes de los individuos del ejercicio 2. Recibe 2 inputs:
# - gene_type: Corresponde al tipo de gen (En este caso es un bit por lo tanto es 'binary' )
# - fact_range: conjunto de valores posibles para un gen (En este caso los valores posibles son 0 o 1)
# Retorna un gen del tipo especificado y dentro del rango especificado.
def gene_factory_ex1(AST, indv_chars):
    allowed_functions = indv_chars['allowed_functions']
    allowed_terminals = indv_chars['allowed_terminals']
    prob_terminal = indv_chars['prob_terminal']
    factory = AST(allowed_functions, allowed_terminals, prob_terminal)    
    return factory

# Función para crear los individuos. Recibe 2 inputs:
# - Función creadora de genes: descrita arriba.
# - Diccionario caracterizando al individuo: diccionario que para cada gen del individuo contiene la información del
#   tipo de gen y del conjunto de valores posibles para ese gen
# Retorna un individuo adecuado para el problema.
def indv_factory(gene_factory_ex1, indv_chars): 
    factory = gene_factory_ex1(AST, indv_chars)
    max_depth = indv_chars['max_depth']
    new_indv = factory(max_depth)   
    return new_indv.copy()  # retornamos el individuo creado


# Función de fitness para el ejercicio 1. Recibe a un individuo y un diccionario con la secuencia y un ponderador.
# Retorna la multiplicación del ponderador y el valor absoluto de la diferencia entre el número entero que representa
# la secuencia entregada y el número entero que representa la secuencia del individuo.
def fitness_ex3(indv, objective):
    val = indv.eval() 
    t_list=[item.eval() for item in indv.serialize() if type(item)==TerminalNode]
    rep_ls=[]
    for item in t_list:
        if t_list.count(item)>1 and item not in rep_ls:
            rep_ls.append(item)
    pen_ls=[t_list.count(item)-1 for item in rep_ls]    
    fitness = objective['ponderador'][0] * abs(objective['secuencia'] - val)+objective['ponderador'][1] * sum(pen_ls)
    return -fitness

In [53]:
# Prueba ejercicio 3
# Seteamos los parametros a ser ocupados por el algoritmo para obtener los gráficos pedidos
secuencia_bits = 65346
ponderador = [1/100,700]
fit_params = {'secuencia': secuencia_bits, 'ponderador': ponderador}
pop_sz_0 = 50
fit_fn = fitness_ex3
cr_genes = gene_factory_ex1
cr_indv = indv_factory
mut_rate = 0.2
term_cond = {'type': 'iterations', 'fitness_th': 0, 'iters': 1000}
selection_type = 'tournament'
slots = 5
elitism_rate = 0.1
random_state = 42
indv_chars = {}
indv_chars['allowed_functions']=[AddNode,SubNode,MultNode]
indv_chars['allowed_terminals']= [25,7,8,100,4,2]
indv_chars['prob_terminal'] =0.3
indv_chars['max_depth']=10
GA = GENALG(pop_sz_0, fit_fn, cr_genes, cr_indv, indv_chars, term_cond, mut_rate, elitism_rate)
generations, goal_cross, overall_max_fitness, overall_fittest_indv = GA.apply(fit_params, selection_type,random_state, slots)
print(overall_max_fitness)
print(overall_fittest_indv)
# Obtener visualización de Evolución de fitness por generación sin elitismo
x = list(generations.keys())
vals = list(generations.values())
max_fit = [item['max_fitness'] for item in vals]
mean_fit = [item['mean_fitness'] for item in vals]
min_fit = [item['min_fitness'] for item in vals]

C:\Users\pclev\Documents\DATA\GIT\Tarea_3\arboles.py:139: RuntimeWarning: overflow encountered in long_scalars
  return x * y
C:\Users\pclev\Documents\DATA\GIT\Tarea_3\arboles.py:102: RuntimeWarning: overflow encountered in long_scalars
  return x + y
C:\Users\pclev\Documents\DATA\GIT\Tarea_3\arboles.py:115: RuntimeWarning: overflow encountered in long_scalars
  return x - y


-4.54
(7 * (((2 - (8 - 100)) * 25) * 4))


In [54]:
overall_fittest_indv.eval()

65800

In [55]:
fig, ax1 = plt.subplots(figsize=(15, 15))
ax1.set_xlabel("Generación", fontsize=15)
ax1.set_ylabel('Fitness (max )')
ax1.plot(x, max_fit, color='green', linewidth=2, label='max_fit')
ax1.legend(loc=2,fontsize=15)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Fitness (mean y min)')  # we already handled the x-label with ax1
ax2.plot(x,  mean_fit, color='blue', linewidth=2, label='mean_fit')
ax2.plot(x, min_fit, color='red', linewidth=2, label='min_fit')
ax2.legend(fontsize=15)
plt.title("Evolución de fitness por generación", fontsize=16, fontweight='bold')
plt.suptitle("Ejercicio 2.1.3", fontsize=20)

plt.savefig('Images\Fitness_por_Generacion_EX3.png')
plt.close()
print('gráfico 3 listo')

gráfico 3 listo


# Ejercicio 4

In [17]:
# Función para crear los genes de los individuos del ejercicio 2. Recibe 2 inputs:
# - gene_type: Corresponde al tipo de gen (En este caso es un bit por lo tanto es 'binary' )
# - fact_range: conjunto de valores posibles para un gen (En este caso los valores posibles son 0 o 1)
# Retorna un gen del tipo especificado y dentro del rango especificado.
def gene_factory_ex1(AST, indv_chars):
    allowed_functions = indv_chars['allowed_functions']
    allowed_terminals = indv_chars['allowed_terminals']
    prob_terminal = indv_chars['prob_terminal']
    factory = AST(allowed_functions, allowed_terminals, prob_terminal)    
    return factory

# Función para crear los individuos. Recibe 2 inputs:
# - Función creadora de genes: descrita arriba.
# - Diccionario caracterizando al individuo: diccionario que para cada gen del individuo contiene la información del
#   tipo de gen y del conjunto de valores posibles para ese gen
# Retorna un individuo adecuado para el problema.
def indv_factory(gene_factory_ex1, indv_chars): 
    factory = gene_factory_ex1(AST, indv_chars)
    max_depth = indv_chars['max_depth']
    new_indv = factory(max_depth)   
    return new_indv.copy()  # retornamos el individuo creado


# Función de fitness para el ejercicio 1. Recibe a un individuo y un diccionario con la secuencia y un ponderador.
# Retorna la multiplicación del ponderador y el valor absoluto de la diferencia entre el número entero que representa
# la secuencia entregada y el número entero que representa la secuencia del individuo.
def fitness_ex4(indv, objective): 
    res=[]
    for x in range(-100,101): 
        vals= {'x':x}
        val=abs(indv.eval(vals=vals)-objective['func'](x))
        res.append(val)      
    fitness = objective['ponderador'] * sum(res)
    return -fitness

In [134]:
# Prueba ejercicio 4
# Seteamos los parametros a ser ocupados por el algoritmo para obtener los gráficos pedidos
def f(x):
    return x**2+x-6
func = f
ponderador = 1/10000
fit_params = {'ponderador': ponderador, 'func':func}
pop_sz_0 = 50
fit_fn = fitness_ex4
cr_genes = gene_factory_ex1
cr_indv = indv_factory
mut_rate = 0.2
term_cond = {'type': 'iterations', 'fitness_th': 0, 'iters': 100}
selection_type = 'tournament'
slots = 5
elitism_rate = 0.1
random_state = 42
indv_chars = {}
indv_chars['allowed_functions']=[AddNode,SubNode,MultNode]
indv_chars['allowed_terminals']= [i for i in range(-10,11)]+['x' for i in range(20)]
indv_chars['prob_terminal'] =0.3
indv_chars['max_depth']=10
GA = GENALG(pop_sz_0, fit_fn, cr_genes, cr_indv, indv_chars, term_cond, mut_rate, elitism_rate)
generations, goal_cross, overall_max_fitness, overall_fittest_indv = GA.apply(fit_params, selection_type,random_state, slots)
print(overall_max_fitness)
print(overall_fittest_indv)
# Obtener visualización de Evolución de fitness por generación sin elitismo
x = list(generations.keys())
vals = list(generations.values())
max_fit = [item['max_fitness'] for item in vals]
mean_fit = [item['mean_fitness'] for item in vals]
min_fit = [item['min_fitness'] for item in vals]

-0.0
((-2 - (x - ((-2 + x) + -2))) - (x * (-2 - (-1 + x))))


In [135]:
fig, ax1 = plt.subplots(figsize=(15, 15))
ax1.set_xlabel("Generación", fontsize=15)
ax1.set_ylabel('Fitness (max )')
ax1.plot(x, max_fit, color='green', linewidth=2, label='max_fit')
ax1.legend(loc=2,fontsize=15)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Fitness (mean y min)')  # we already handled the x-label with ax1
ax2.plot(x,  mean_fit, color='blue', linewidth=2, label='mean_fit')
ax2.plot(x, min_fit, color='red', linewidth=2, label='min_fit')
ax2.legend(fontsize=15)
plt.title("Evolución de fitness por generación", fontsize=16, fontweight='bold')
plt.suptitle("Ejercicio 2.3", fontsize=20)

plt.savefig('Images\Fitness_por_Generacion_EX4.png')
plt.close()
print('gráfico 4 listo')

gráfico 4 listo


# Ejercicio 5

In [5]:
# Función para crear los genes de los individuos del ejercicio 2. Recibe 2 inputs:
# - gene_type: Corresponde al tipo de gen (En este caso es un bit por lo tanto es 'binary' )
# - fact_range: conjunto de valores posibles para un gen (En este caso los valores posibles son 0 o 1)
# Retorna un gen del tipo especificado y dentro del rango especificado.
def gene_factory_ex1(AST, indv_chars):
    allowed_functions = indv_chars['allowed_functions']
    allowed_terminals = indv_chars['allowed_terminals']
    prob_terminal = indv_chars['prob_terminal']
    factory = AST(allowed_functions, allowed_terminals, prob_terminal)    
    return factory

# Función para crear los individuos. Recibe 2 inputs:
# - Función creadora de genes: descrita arriba.
# - Diccionario caracterizando al individuo: diccionario que para cada gen del individuo contiene la información del
#   tipo de gen y del conjunto de valores posibles para ese gen
# Retorna un individuo adecuado para el problema.
def indv_factory(gene_factory_ex1, indv_chars): 
    factory = gene_factory_ex1(AST, indv_chars)
    max_depth = indv_chars['max_depth']
    new_indv = factory(max_depth)   
    return new_indv.copy()  # retornamos el individuo creado


# Función de fitness para el ejercicio 1. Recibe a un individuo y un diccionario con la secuencia y un ponderador.
# Retorna la multiplicación del ponderador y el valor absoluto de la diferencia entre el número entero que representa
# la secuencia entregada y el número entero que representa la secuencia del individuo.
def fitness_ex5(indv, objective): 
    res=[]
    infeasible= 0
    for x in range(-100,101): 
        vals= {'x':x}
        try:
            val=abs(indv.eval(vals=vals)-objective['func'](x))
        except:
            infeasible =1
            val=100000
        res.append(val)      
    fitness = objective['ponderador'][0] * sum(res)+objective['ponderador'][1]*infeasible
    return -fitness

In [6]:
# Prueba ejercicio 5
# Seteamos los parametros a ser ocupados por el algoritmo para obtener los gráficos pedidos
def f(x):
    return x**2+x-6
func = f
ponderador = [1/1000000,1000000]
fit_params = {'ponderador': ponderador, 'func':func}
pop_sz_0 = 50
fit_fn = fitness_ex5
cr_genes = gene_factory_ex1
cr_indv = indv_factory
mut_rate = 0.2
term_cond = {'type': 'iterations', 'fitness_th': 0, 'iters': 100}
selection_type = 'tournament'
slots = 5
elitism_rate = 0.1
random_state = 42
indv_chars = {}
indv_chars['allowed_functions']=[AddNode,SubNode,MultNode,DivNode]
indv_chars['allowed_terminals']= [i for i in range(-10,11)]+['x' for i in range(20)]
indv_chars['prob_terminal'] =0.3
indv_chars['max_depth']=10



In [10]:
GA = GENALG(pop_sz_0, fit_fn, cr_genes, cr_indv, indv_chars, term_cond, mut_rate, elitism_rate)
generations, goal_cross, overall_max_fitness, overall_fittest_indv = GA.apply(fit_params, selection_type,random_state, slots)

In [11]:
print(overall_max_fitness)
print(overall_fittest_indv)
# Obtener visualización de Evolución de fitness por generación sin elitismo
x = list(generations.keys())
vals = list(generations.values())
max_fit = [item['max_fitness'] for item in vals]
mean_fit = [item['mean_fitness'] for item in vals]
min_fit = [item['min_fitness'] for item in vals]

-0.0
(-6 + (x + (x * x)))


In [12]:
fig, ax1 = plt.subplots(figsize=(15, 15))
ax1.set_xlabel("Generación", fontsize=15)
ax1.set_ylabel('Fitness (max )')
ax1.plot(x, max_fit, color='green', linewidth=2, label='max_fit')
ax1.legend(loc=2,fontsize=15)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Fitness (mean y min)')  # we already handled the x-label with ax1
ax2.plot(x,  mean_fit, color='blue', linewidth=2, label='mean_fit')
ax2.plot(x, min_fit, color='red', linewidth=2, label='min_fit')
ax2.legend(fontsize=15)
plt.title("Evolución de fitness por generación", fontsize=16, fontweight='bold')
plt.suptitle("Ejercicio 2.4.3", fontsize=20)

plt.savefig('Images\Fitness_por_Generacion_EX5.png')
plt.close()
print('gráfico 5 listo')

gráfico 5 listo


In [7]:
overall_fittest_indv.isfeasible()

True

In [7]:
b=indv_factory(gene_factory_ex1, indv_chars)

In [8]:
b

((((x * 4) + (((((x - -6) + (((-3 + x) + (-3 / x)) / ((7 + x) + 7))) + (((4 / (x * 2)) - ((x / x) - -2)) / ((x / x) * (x - (-5 * -7))))) * x) + (((-5 + (x * ((2 * x) - (x + -1)))) / (x * ((x - (10 + -4)) - ((x - 5) * (-8 / x))))) * (x / ((7 / (x + -8)) - (-4 + x)))))) * (x / 2)) - (x + (-1 / (x - ((((-7 * ((-4 / x) / (-6 - x))) * (((x * 1) * x) * x)) + ((((5 / -3) / (2 * -1)) * (x - 4)) * (7 + x))) + (x + ((((-1 + 2) - (x / -6)) + (8 - (4 + x))) / ((x * (3 + 9)) + ((-9 / x) + (x + x))))))))))

In [9]:
b.eval({'x':0})

ZeroDivisionError: float division by zero

In [25]:
b.isfeasible()

0

In [10]:
b.eval({'x':0})

-0.0

In [19]:
b.serialize()

[(((-7 * -3) - ((-4 + 2) * (((x / 8) * (((x * -6) - (((4 + 10) / -3) * ((x * -1) * 10))) - (((x * x) - ((x + 5) / (6 + x))) - x))) - (((((x / (x / 10)) / 2) + (((x * 7) / (x * -8)) / 4)) / ((((1 - x) + (x * -3)) * ((-6 / 0) / (-7 - 3))) + ((1 / (x * x)) + ((x / x) + (-5 / x))))) / (-4 - (((-3 - x) - ((x * x) * (2 / -8))) - ((1 + 9) / (x - (-3 / 5))))))))) + x),
 ((-7 * -3) - ((-4 + 2) * (((x / 8) * (((x * -6) - (((4 + 10) / -3) * ((x * -1) * 10))) - (((x * x) - ((x + 5) / (6 + x))) - x))) - (((((x / (x / 10)) / 2) + (((x * 7) / (x * -8)) / 4)) / ((((1 - x) + (x * -3)) * ((-6 / 0) / (-7 - 3))) + ((1 / (x * x)) + ((x / x) + (-5 / x))))) / (-4 - (((-3 - x) - ((x * x) * (2 / -8))) - ((1 + 9) / (x - (-3 / 5))))))))),
 (-7 * -3),
 -7,
 -3,
 ((-4 + 2) * (((x / 8) * (((x * -6) - (((4 + 10) / -3) * ((x * -1) * 10))) - (((x * x) - ((x + 5) / (6 + x))) - x))) - (((((x / (x / 10)) / 2) + (((x * 7) / (x * -8)) / 4)) / ((((1 - x) + (x * -3)) * ((-6 / 0) / (-7 - 3))) + ((1 / (x * x)) + ((x / x) + (-5

In [28]:
b.serialize()[88]

(-5 / x)

In [29]:
f=b.serialize()[88]

In [30]:
f.isfeasible()

True

In [31]:
f.eval({'x':0})

0

In [32]:
f.isfeasible()

True

In [12]:
b.isfeasible()

True

In [145]:
fig, ax1 = plt.subplots(figsize=(15, 15))
ax1.set_xlabel("Generación", fontsize=15)
ax1.set_ylabel('Fitness (max )')
ax1.plot(x, max_fit, color='green', linewidth=2, label='max_fit')
ax1.legend(loc=2,fontsize=15)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Fitness (mean y min)')  # we already handled the x-label with ax1
ax2.plot(x,  mean_fit, color='blue', linewidth=2, label='mean_fit')
ax2.plot(x, min_fit, color='red', linewidth=2, label='min_fit')
ax2.legend(fontsize=15)
plt.title("Evolución de fitness por generación", fontsize=16, fontweight='bold')
plt.suptitle("Ejercicio 2.4.3", fontsize=20)

plt.savefig('Images\Fitness_por_Generacion_EX5.png')
plt.close()
print('gráfico 5 listo')

gráfico 5 listo
